# Human Pose Estimation With MET Score

## Preprocessing Code

학습용으로 사용하기 위한 이미지 데이터와 좌푯값 전처리 코드.

* CSV 구성 형식

  이미지 경로 , 관절좌표1, 관절좌표2, ... , 관절좌표14, MET Score
  
* 처리 형식

    1. 전체 데이터세트 이미지 최대 크기와 평균 크기 측정 -> 이미지 크기 조정
    2. 크기 조정 비율에 따른 좌표값 재처리
    
* 추가 안내

해당 코드를 실행하면서 Jupyter Notebook 상으로 사진을 확인하면 이미지의 색이 이상하게 보이는 현상이 있음. 
matplotlib과 cv2의 색 채널을 서로 다르게 사용하기 때문임.

_**matplitlib은 RGB로 보여주지만 opencv는 채널이 BGR로 사용함**_

그래도 전처리 할 때에는 이 채널을 transpose 하여 하기 때문에 이미지 저장을 할 때는 크게 신경쓰지 않고 저장하여도 상관 없음.

In [1]:
import numpy as np # Numpy 배열 사용용도
import matplotlib.pyplot as plt # 이미지출력 용도
import pandas as pd # csv 파일을 pandas 프레임으로 호출 용도
import cv2 # 이미지 불러오기 및 저장용도

# 주피터 노트북 상에 이미지를 출력하도록 설정
%matplotlib inline

In [2]:
# dataset CSV 파일을 불러와 pandas 프레임으로 저장한 뒤에 Numpy형식의 행렬로 변환하여 testset 변수에 저장
testset=pd.read_csv("/home/hahnz/dataset.csv",header=None).as_matrix()

In [3]:
# 변수 초기화.
# 모든 x축 크기 총합, 모든 y축 크기 총합, 최대 x축 크기, 최대 y축 크기
allsum_x=allsum_y=max_x=max_y=0

for i in range(len(testset)): # testset 길이만큼 반복 
    img=cv2.imread(testset[i][0]) # 이미지 불러오기
    if max_x<img.shape[0]: max_x=img.shape[0] # 최대 x축 길이값 저장
    if max_y<img.shape[1]: max_y=img.shape[1] # 최대 y축 길이값 저장
    allsum_x+=img.shape[0] # 모든 이미지의 x축 크기 총합
    allsum_y+=img.shape[1] # 모든 이미지의 y축 크기 총합
avg_x=allsum_x/len(testset) # 전체 이미지의 x축 크기 평균
avg_y=allsum_y/len(testset) # 전체 이미지의 x축 크기 평균
print (avg_x,avg_y,max_x,max_y) # 모든 x축 크기 총합, 모든 y축 크기 총합, 최대 x축 크기, 최대 y축 크기 출력

221.51 243.43 769 834


**평균 값** -> **x**: 221.51, **y**:243.43

이미지 크기를 256 x 256 으로 조정

In [4]:
# 문자열의 좌표값을 정수형 좌표값으로 변환하는 함수
def convert_str_to_int(coors):
    coor=np.zeros([len(coors),14,2],dtype=int) # 변환한 좌표를 저장하기 위한 배열 초기화
    for i in range(len(coors)): # 주어진 갯수 만큼 반복
        for j in range(14): # 좌표 개수 만큼 반복
            a=0 # 컴마 위치 확인용
            # "," 위치 체크
            while True: # 무한 반복문
                if coors[i][j+1][a]=="," : break # 해당 문자가 컴마면 무한 반복문 탈출
                else : a+=1 # 컴마가 아니면 다음 문자로 넘어감
            # ,를 기준으로 각각의 좌표 x y를 나누어 int로 형변환 하여 저장
            coor[i][j]=int(coors[i][j+1][1:a]),int(coors[i][j+1][a+1:-1])
    # 이렇게 반복하여 추출한 좌표값을 반환
    return coor

In [5]:
def Preprocess(dataset):
    img_set=np.zeros([len(dataset),256,256,3]) # 이미지를 저장해두기 위한 텐서 배열 초기화 - 이미지 갯수 / x크기 / y크기 / RGB
    coor_set=convert_str_to_int(dataset) # 문자열의 좌표값을 정수형 좌표값으로 변환하여 저장
    
    for i in range(len(dataset)): # 주어진 데이터 갯수만큼 반복
        img=cv2.imread(dataset[i][0], cv2.COLOR_BGR2RGB) # 이미지 불러오기
        
        img_set[i]=cv2.resize(img,(256,256)) # 이미지 크기를 재조정 하여 저장
        
        #이미지 크기 변환을 하였으므로 그에 걸맞게 좌표 위치또한 재조정
        for j in range(14): #주어진 좌표 개수 만큼 반복
            if coor_set[i][j][0]==-1: coor_set[i][j][0]=-1 # x 좌표 값이 -1 이면 조정을 해주지 않음
            else: coor_set[i][j][0]=coor_set[i][j][0]*(256/img.shape[1]) # x축 크기의 변환된 비율 만큼 곱해 Scaling 
            if coor_set[i][j][1]==-1: coor_set[i][j][1]=-1 # y 좌표 값이 -1 이면 조정을 해주지 않음
            else: coor_set[i][j][1]=coor_set[i][j][1]*(256/img.shape[0]) # y축 크기의 변환된 비율 만큼 곱해 Scaling 
    
    # 이렇게 변환한 이미지 세트와 좌표 세트를 반환
    return img_set, coor_set

In [6]:
# 전처리 함수의 입력으로 불러온 CSV 파일을 저장한 testset 변수로 입력.
# 변환된 이미지 세트와 좌표 세트를 각각 picpic과 coco에다 저장해줌.
picpic,coco=Preprocess(testset)

In [7]:
# 각 세트의 크기 확인
print (picpic.shape,coco.shape)

(100, 256, 256, 3) (100, 14, 2)


In [8]:
# 이미지와 좌표를 받아와서 이미에 해당 좌표를 입력
# 잘 변환이 되었는지 확인하는 용도
def markJoints(img, joints):  
    numJoints, t = joints.shape
    circSize=10
    font = cv2.FONT_HERSHEY_SIMPLEX
    for i in range(numJoints):
        x = int(joints[i,0])
        y = int(joints[i,1])
    
        cv2.circle(img, (x, y), 4, (255, 0, 0), thickness=-1)
        cv2.putText(img, str(i+1), (x,y), font, 0.5, (100,100,100), 1, cv2.LINE_AA)
    return img

In [9]:
# 전처리된 이미지 저장
%mkdir p_img # 현재 디렉토리에 저장할 디렉토리 생성
for i in range(len(picpic)):
    cv2.imwrite("./processed_img/p-img_"+str(i)+".jpg",picpic[i])

In [10]:
# 전처리된 이미지에 좌표를 찍어서 저장
%mkdir p_with_coors_img # 현재 디렉토리에 저장할 디렉토리 생성
for i in range(len(picpic)):
    cv2.imwrite("./processed_img/p-img_"+str(i)+".jpg",markJoints(picpic[i],coco[i]))